# 2: Regular Expressions, Text Normalization, Edit Distance

**2.1**: Write regular expressions for the following languages.


1.   the set of all alphabetic strings;
2.   the set of all lower case alphabetic strings ending in a *b*
3.   the set of all strings from the alphabet *a, b* such that each *a* is immediately preceded by and immediately followed by a *b*;



In [94]:
import re

strings = ["a_b0c", "Db", "AbcdE", "bbabbbabbb", "abab", "123", "1b", "bb"]

# We interpret "string" as an entire word in 'strings', a list of words
print("1. The set of all alphabetic strings:")
for s in strings:
  match_1 = re.search(r'^[a-zA-Z]+$', s)
  if match_1:
    print(' ', match_1.group())
print('\n')

print("2. The set of all lower-case alphabetic strings ending in a 'b':")
for s in strings:
  match_2 = re.search(r'^[a-z]*b$', s)
  if match_2:
    print(' ', match_2.group())
print('\n')

print("3. The set of all strings from the alphabet [a,b] such that\n",\
      "each a is immediately preceded by and immediately followed by a b:")
for s in strings:
  match_3 = re.search(r'^b+(?:ab+)*b*$', s)
  if match_3:
    print(' ', match_3.group())

1. The set of all alphabetic strings:
  Db
  AbcdE
  bbabbbabbb
  abab
  bb


2. The set of all lower-case alphabetic strings ending in a 'b':
  bbabbbabbb
  abab
  bb


3. The set of all strings from the alphabet [a,b] such that
 each a is immediately preceded by and immediately followed by a b:
  bbabbbabbb
  bb


**2.2**: Write regular expressions for the following languages. By "word", we mean an alphabetic string separated from other words by whitespace, any relevant punctuation, line breaks, and so forth.


1.   the set of all strings with two consecutive repeated words (e.g., "Humbert Humbert" and "the the" but not "the bug" or "the big bug");
2.   all strings that start at the beginning of the line with an integer and that end at the end of the line with a word;
3.   all strings that have both the word *grotto* and the word *raven* in them (but not, e.g., words llike *grottos* that merely *contain* the word *grotto*);
4.   write a pattern that places the first word of an English sentence in a register. Deal with punctuation.



In [97]:
import re

strings = ["Humbert Humbert", "thethe", "the big bug", "the bug bug",\
         "1,one", "2a two", "3:three", "4four", "5 4five", "z0 zero",\
         "grotto raven", "grottos raven", "grottoraven", "h raven grotto h"]

print("1. The set of all strings with two consecutive repeated words:")
for s in strings:
  match_1 = re.search(r'(?:^|^.+\W)([a-zA-Z]+)\W+\1(?:$|\W.+$)', s)
  if match_1:
    print(' ', match_1.group())
print('\n')

print("2. All strings that start at the beginning of the line with an",\
      "integer and that end at the end of the line with a word:")
for s in strings:
  match_2 = re.search(r'^\d.*\W[a-zA-Z]+$', s)
  if match_2:
    print(' ', match_2.group())
print('\n')

print("3. All strings that have both the word grotto and the word raven",\
      "in them:")
for s in strings:
  match_3 = re.search(r'(^|^.+\W)(grotto\W.*raven|raven\W.*grotto)($|\W.+$)', s)
  if match_3:
    print(' ', match_3.group())
print('\n')

print("4. Write a pattern that places the first word of an English sentence",\
      "in a register. Deal with punctuation.")
sentence = "!!Hello, world: this is a regular expression!"
match_4 = re.search(r'^\W*([a-zA-Z]+)(?:$|\W.+$)', sentence)
if match_4 and match_4.group(1):
  # Print the first captured group in the register
  print(' The first word of sentence: "', sentence, '" is: "',\
        match_4.group(1), '"', sep="")

1. The set of all strings with two consecutive repeated words:
  Humbert Humbert
  the bug bug


2. All strings that start at the beginning of the line with an integer and that end at the end of the line with a word:
  1,one
  2a two
  3:three


3. All strings that have both the word grotto and the word raven in them:
  grotto raven
  h raven grotto h


4. Write a pattern that places the first word of an English sentence in a register. Deal with punctuation.
 The first word of sentence: "!!Hello, world: this is a regular expression!" is: "Hello"


**2.3**: Implement an ELIZA-like program, using substitutions such as those described on page 11. You might want to choose a different domain than a Rogerian psychologist, although keep in mind that you would need a domain in which your program can legitimately engage in a lot of simple repetition.